In [3]:
import warnings
warnings.filterwarnings('error')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.")


import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the diabetes dataset
data = load_diabetes()

# Create a DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)

# List containing the range of the percentages of missing data
missing_data_percentages = [5, 10, 15, 20, 25]

# Number of iterations the imputation model will run for
num_iterations = 5

# Create an empty DataFrame called to store the performance metric results
results_df = pd.DataFrame(columns=["Missing Percentage", "Iteration", "MAE", "MSE", "RMSE"])

# Iterate through each missing percentage of data
for missing_percentage in missing_data_percentages:
    # Empty lists to contain the performance metric for each of the iterations
    mae_list = []
    mse_list = []
    rmse_list = []

    for _ in range(num_iterations):

        # Calculate the number of rows to remove
        rows_to_remove = int(df.shape[0] * (missing_percentage / 100))

        # Create a duplicate of the orignal DataFrame 
        df_with_missing = df.copy()

        # Iterate over each column and remove the specified percentage of the values within that column randomly
        # The same number of data points in each column will be removed
        for col in df.columns:
            random_indices = np.random.choice(df.shape[0], size=rows_to_remove, replace=False)
            df_with_missing.loc[random_indices, col] = np.nan


        # Impute the missing values using Linear Regression
        linearReg_imputer = IterativeImputer(estimator=LinearRegression(), max_iter=350, random_state=100)
        df_imputed = linearReg_imputer.fit_transform(df_with_missing)

        # Convert the newly imputed values into a DataFrame
        df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

        # Calculate the MAE, MSE and RMSE between the newly imputed values and the original values
        mae = mean_absolute_error(df.values, df_imputed.values)
        mse = mean_squared_error(df.values, df_imputed.values)
        rmse = np.sqrt(mse)

        # Append the metrics to their lists
        mae_list.append(mae)
        mse_list.append(mse)
        rmse_list.append(rmse)

    # Calculate the average metrics for the current percentage of missing data in the dataset
    average_mae = np.mean(mae_list)
    average_mse = np.mean(mse_list)
    average_rmse = np.mean(rmse_list)

    # Create a DataFrame for the current percentage of missing data
    results_percentage_df = pd.DataFrame({
        "Missing Percentage": [missing_percentage] * num_iterations,
        "Iteration": range(1, num_iterations + 1),
        "MAE": mae_list,
        "MSE": mse_list,
        "RMSE": rmse_list
    })

    # Append the average metrics calculated earlier to the DataFrame
    results_percentage_df.loc[len(results_percentage_df.index)] = [missing_percentage, "Average", average_mae, average_mse, average_rmse]

    # Append results for the current percentage of missing data to the overall results DataFrame
    results_df = pd.concat([results_df, results_percentage_df], ignore_index=True)
    
    

# Display the results
print("Results:")
print(display(results_df))

Results:


,Missing Percentage,Iteration,MAE,MSE,RMSE
0,5,1,0.001125,0.000051,0.007114
1,5,2,0.000976,0.000035,0.005946
2,5,3,0.001092,0.000051,0.007163
3,5,4,0.001109,0.000049,0.006989
4,5,5,0.000986,0.000042,0.006478
5,5,Average,0.001058,0.000046,0.006738
6,10,1,0.002324,0.000100,0.009989
7,10,2,0.002241,0.000095,0.009765
8,10,3,0.002202,0.000097,0.009862
9,10,4,0.002317,0.000104,0.010189


None
